In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/sign-language-mnist/sign_mnist_train.csv')
test_data = pd.read_csv('../input/sign-language-mnist/sign_mnist_test.csv')

In [ ]:
train_data.head()

In [ ]:
#convert the dataframe into array

training = train_data.iloc[:,:].values
test = test_data.iloc[:,:].values

In [ ]:
#saperate label (column: 1) in both training and test data and turn 784 pixels into 28x28
train_labels = training[:,0]
train_images = training[:,1:]
test_labels = test[:,0]
test_images = test[:, 1:]

#Reshape the data --> This is the form CNN will accept the data
train_images = train_images.reshape(train_images.shape[0], *(28, 28, 1))
test_images = test_images.reshape(test_images.shape[0], *(28, 28, 1))

In [ ]:
print(train_images.shape)
print(test_images.shape)

In [ ]:
import random
import matplotlib.pyplot as plt
#Randomly check some image from the file
#Run this block multiple times to see different images
i = random.randint(1, 27456)
plt.imshow(training[i, 1:].reshape(28,28))
label = training[i, 0]
print(label)

In [ ]:
# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True
                                  )

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
#Define the Modle
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax'),
])

model.summary()

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_gen = train_datagen.flow(train_images,
                               train_labels,
                              batch_size=64,
                              )

test_gen = test_datagen.flow(test_images,
                            test_labels,
                            batch_size=32,
                            )

In [ ]:
#Train the model
history = model.fit(train_gen,
                   epochs=30,
                   validation_data=test_gen
                   )

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()